# Feature Engineering with SHAP values Experiment 1

SHAP Images of server with different number of poisoned clients
* number of malicious clients [0,1,2,3,4,5]

SHAP Images right after poisoning attack
* rounds [1,2,10,75,200]

Summation of Differences SHAP Images right after poisoning attack
* rounds [1,2,10,75,200]
* 5 times

## Google Colab

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
sys.path.append('/content/drive/My Drive/Colab Notebooks/federated_learning')

!pip install shap==0.40.0

## Experiments

In [ ]:
from federated_learning.utils import SHAPUtil, experiment_util, Visualizer
from federated_learning import ClientPlane, Configuration, ObserverConfiguration
from federated_learning.server import Server

# MNIST
(1) 5 → 4,
(2) 1 → 7,
(3) 3 → 8,

In [ ]:
from federated_learning.nets import MNISTCNN
from federated_learning.dataset import MNISTDataset
import os
config = Configuration()
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = MNISTDataset
config.MODELNAME = config.MNIST_NAME
config.NETWORK = MNISTCNN
observer_config = ObserverConfiguration()
neutral_label = 2

In [ ]:
# Google Colab Settigns
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"

In [ ]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)
visualizer = Visualizer(shap_util)

In [ ]:
import numpy as np
import copy
import torch
for i in range(200):
    experiment_util.set_rounds(client_plane, server, i+1)
    experiment_util.run_round(client_plane, server, i+1)
print("Run 199 finished")
old_params = copy.deepcopy(server.get_nn_parameters())
file = "./temp/models/ex2/MNIST_round_199.model"
if not os.path.exists(os.path.dirname(file)):
        os.makedirs(os.path.dirname(file))
torch.save(server.net.state_dict(), file)

### alpha (5,4)

In [ ]:
config.FROM_LABEL = 5
config.TO_LABEL = 4

accuracy_list = []
recall_list = []

In [ ]:
server.net =  MNISTCNN()
server.net.load_state_dict(torch.load('./temp/models/ex2/MNIST_round_199.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

for j in range(config.CLIENTS_PER_ROUND + 1):
    client_plane.reset_default_client_nets()
    server.update_nn_parameters(old_params)
    file_name = './results/ex2/MNIST/5_4/_run_shap_values_{}_poisoned_clients_alpha_5_4.pdf'.format(j)
    
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracy_list.append(accuracy)
    recall_list.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    
    client_plane.reset_poisoning_attack()

In [ ]:
# Run 0
print(accuracy_list)
print(recall_list)

# FashionMNIST
For Fashion-MNIST we experiment with 
(1) 5: sandal → 4: coat,
(2) 1: trouser → 3: dress, and 
(3) 8: Bag → 9: Ankle Boot.
['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker',  'Bag', 'Ankle Boot']


In [ ]:
from federated_learning.nets import FMNISTCNN
from federated_learning.dataset import FMNISTDataset
import os
config = Configuration()
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = FMNISTDataset
config.MODELNAME = config.FMNIST_NAME
config.NETWORK = FMNISTCNN
observer_config = ObserverConfiguration()
neutral_label = 2

In [ ]:
# Google Colab Settigns
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/mnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/cifar10')
config.VM_URL = "none"

In [ ]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)
visualizer = Visualizer(shap_util)

In [ ]:
import numpy as np
import copy
import torch
for i in range(200):
    experiment_util.set_rounds(client_plane, server, i+1)
    experiment_util.run_round(client_plane, server, i+1)
print("Run 199 finished")
old_params = copy.deepcopy(server.get_nn_parameters())
file = "./temp/models/ex2/FMNIST_round_199.model"
if not os.path.exists(os.path.dirname(file)):
    os.makedirs(os.path.dirname(file))
torch.save(server.net.state_dict(), file)

## Experiment

In [ ]:
from federated_learning.nets import FMNISTCNN
import torch
import copy
config.FROM_LABEL = 5
config.TO_LABEL = 4

server.net =  FMNISTCNN()
server.net.load_state_dict(torch.load('./temp/models/ex2/FMNIST_round_199.model'))
old_params = copy.deepcopy(server.get_nn_parameters())

accuracy_list = []
recall_list = []

In [ ]:
for j in range(config.CLIENTS_PER_ROUND + 1):
    client_plane.reset_default_client_nets()
    server.update_nn_parameters(old_params)
    file_name = './results/ex3/FMNIST/5_4/3_run_shap_values_{}_poisoned_clients_alpha_5_4.pdf'.format(j)
    
    config.POISONED_CLIENTS = j
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    clean_clients = experiment_util.select_random_clean(client_plane, config, config.CLIENTS_PER_ROUND - j)
    poisoned_clients = experiment_util.select_poisoned(client_plane, j)
    clients = [*clean_clients, *poisoned_clients]
    experiment_util.run_round_with(clients, old_params, client_plane, server, 200)
    
    server.test()
    server_shap = server.get_shap_values()
    recall, precision, accuracy = server.analize_test()
    accuracy_list.append(accuracy)
    recall_list.append(recall)
    visualizer.plot_shap_values(server_shap,file_name)
    
    client_plane.reset_poisoning_attack()

In [ ]:
print(accuracy_list)
print(recall_list)